# 1 Load Packages

In [1]:
import xarray as xr
from pathlib import Path
import tensorflow as tf
import pickle
import logging

logging.basicConfig(level=logging.DEBUG)

In [2]:
import xarray_tfrecords

# 2 Load Example File

In [3]:
data = xr.load_dataset("/media/bjoern/69541ddc-6387-4b54-867e-d386a7abc282/Data/CMIP6/data/CMIP/MPI-M/MPI-ESM1-2-LR/historical/r10i1p1f1/Omon/tos/gn/v20190710/tos_Omon_MPI-ESM1-2-LR_historical_r10i1p1f1_gn_185001-186912.nc", use_cftime=True)

In [4]:
data_sum = data.sum(dim=("i","j"))

In [5]:
data_sum["tos"].rename("sum")
data_com = xr.merge([data, data_sum["tos"].rename("sum")])

In [6]:
data_path = os.path.join("..","data","data.tfrecord")

In [7]:
import importlib
importlib.reload(xarray_tfrecords)

<module 'xarray_tfrecords' from '/home/bjoern/anaconda3/envs/xr-tf-inst/lib/python3.9/site-packages/xarray_tfrecords/__init__.py'>

In [8]:
xarray_tfrecords.xarray_to_tfrecord(data_com, sample_dim="time",feature_variables=["tos","sum"], data_path=data_path)

INFO:root:0
INFO:root:1
INFO:root:2
INFO:root:3
INFO:root:4
INFO:root:5
INFO:root:6
INFO:root:7
INFO:root:8
INFO:root:9
INFO:root:10
INFO:root:11
INFO:root:12
INFO:root:13
INFO:root:14
INFO:root:15
INFO:root:16
INFO:root:17
INFO:root:18
INFO:root:19
INFO:root:20
INFO:root:21
INFO:root:22
INFO:root:23
INFO:root:24
INFO:root:25
INFO:root:26
INFO:root:27
INFO:root:28
INFO:root:29
INFO:root:30
INFO:root:31
INFO:root:32
INFO:root:33
INFO:root:34
INFO:root:35
INFO:root:36
INFO:root:37
INFO:root:38
INFO:root:39
INFO:root:40
INFO:root:41
INFO:root:42
INFO:root:43
INFO:root:44
INFO:root:45
INFO:root:46
INFO:root:47
INFO:root:48
INFO:root:49
INFO:root:50
INFO:root:51
INFO:root:52
INFO:root:53
INFO:root:54
INFO:root:55
INFO:root:56
INFO:root:57
INFO:root:58
INFO:root:59
INFO:root:60
INFO:root:61
INFO:root:62
INFO:root:63
INFO:root:64
INFO:root:65
INFO:root:66
INFO:root:67
INFO:root:68
INFO:root:69
INFO:root:70
INFO:root:71
INFO:root:72
INFO:root:73
INFO:root:74
INFO:root:75
INFO:root:76
INFO:root

In [9]:
data_loaded = xarray_tfrecords.tfrecord_to_xarray("../data/data.tfrecord")

2022-03-04 19:20:21.610377: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-03-04 19:20:21.610576: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-03-04 19:20:21.611631: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.
INFO:root:tos Variable loading started
2022-03-04 19:20:21.844576: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-03-04 19:20:21.862336: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2299

# 3 Load TFRecord File with tensorflow and train model

In [10]:
def decode_feature(feature):
    # Reshape and drop nan values
    feature = tf.reshape(feature,[-1,1])
    feature_mask = ~tf.math.is_nan(feature)
    feature = tf.boolean_mask(feature,feature_mask)
    return feature


In [11]:
def read_tfrecord(example, feature_description):
    example = tf.io.parse_single_example(example, feature_description)
    #return example
    feature = decode_feature(example["tos"])
    target = example["sum"]

    return feature, target


In [12]:
def xr_load_tfdataset(filename, sample_dim):
    dataset = tf.data.TFRecordDataset(filename)

    metadata = xarray_tfrecords.load_metadata("../data/data.tfrecord")
    feature_description = xarray_tfrecords.gen_feature_description(metadata, sample_dim=sample_dim)
    
    dataset_parsed = dataset.map(lambda example : read_tfrecord(example, feature_description))
    return dataset_parsed

In [13]:
dataset_parsed = xr_load_tfdataset("../data/data.tfrecord", sample_dim="time")

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [14]:
dataset_parsed = dataset_parsed.shuffle(10)
dataset_parsed = dataset_parsed.batch(10)

In [21]:
n_feature = iter(dataset_parsed).next()[0].shape[1]

In [24]:
input = tf.keras.layers.Input(shape=(n_feature,))
output = tf.keras.layers.Dense(1, activation="relu")(input)

In [25]:
model = tf.keras.Model(inputs=input, outputs=output)

In [26]:
model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = 1e-5),loss="mean_squared_error")

In [27]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 36791)]           0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 36792     
Total params: 36,792
Trainable params: 36,792
Non-trainable params: 0
_________________________________________________________________


In [28]:
history = model.fit(dataset_parsed, epochs=100)

Epoch 1/100
24/24 [==============================] - 1s 17ms/step - loss: 120733844111.3600
Epoch 2/100
24/24 [==============================] - 0s 16ms/step - loss: 120622255308.8000
Epoch 3/100
24/24 [==============================] - 0s 14ms/step - loss: 120267768791.0400
Epoch 4/100
24/24 [==============================] - 1s 17ms/step - loss: 120213312634.8800
Epoch 5/100
24/24 [==============================] - 0s 14ms/step - loss: 120382304092.1600
Epoch 6/100
24/24 [==============================] - 0s 11ms/step - loss: 120528084664.3200
Epoch 7/100
24/24 [==============================] - 0s 12ms/step - loss: 120272930734.0800
Epoch 8/100
24/24 [==============================] - 0s 13ms/step - loss: 120341143879.6800
Epoch 9/100
24/24 [==============================] - 0s 12ms/step - loss: 120108948848.6400
Epoch 10/100
24/24 [==============================] - 0s 13ms/step - loss: 119989378416.6400
Epoch 11/100
24/24 [==============================] - 0s 13ms/step - loss: 1201